# Retrieval Augmented Generation (RAG)
<sup>This notebook is a part of Natural Language Processing class at the University of Ljubljana, Faculty for computer and information science. Please contact [boshko.koloski@ijs.si](mailto:boshko.koloski@ijs.si) for any comments.</sub>




The core functionality of this notebook is to create a retrieval-augmented generation (RAG) system that enables discussion about the NLP subject setting using the `Mistral-7b-v0.2` model.

General-purpose language models can be fine-tuned for common tasks such as sentiment analysis and named entity recognition, which typically do not require additional background knowledge.

For more complex and knowledge-intensive tasks, it is possible to construct a language model-based system that accesses external knowledge sources. This approach enhances factual consistency, improves the reliability of responses, and mitigates the issue of 'hallucination.'

Researchers have introduced the Retrieval Augmented Generation (RAG) method to address these knowledge-intensive tasks. RAG integrates an information retrieval component with a text generator model, allowing for efficient updates and modifications to its internal knowledge without retraining the entire model.

RAG operates by taking an input, retrieving a set of relevant or supporting documents from a source like Wikipedia, and concatenating these documents with the original input prompt. This concatenated context is then fed to the text generator, which produces the final output. This adaptability is crucial for situations where facts may evolve over time, as the static parametric knowledge of traditional large language models (LLMs) can become outdated. RAG circumvents the need for retraining, providing access to the most current information and enabling reliable outputs through retrieval-based generation.

RAG requires additional document embeddings and the storage of documents in a database for retrieval purposes.


### Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `bitsandbytes`, `accelerate`, `transformers`, `datasets`, `sentence-transformers` and  `faiss-gpu`.

In [ ]:
!pip install langchain bitsandbytes accelerate sentence-transformers faiss-cpu langchain_community unstructured python-docx --user

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cu121

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())           # should be True
print(torch.version.cuda)                  # should show 12.1
print(torch.cuda.get_device_name(0))       # should return your GPU name


2.2.0+cu121
True
12.1
NVIDIA GeForce RTX 4090


**In simple terms, RAG is to LLMs what an open-book exam is to humans.**

The concept of an open-book exam centers around assessing a student's reasoning abilities rather than their capacity to memorize specific details. In a similar vein, RAG separates factual knowledge from the LLM’s reasoning capabilities. This factual information is stored in an external knowledge source, which is both easily accessible and updatable:

- **Parametric knowledge:** Knowledge that is learned during training and implicitly stored within the neural network's weights.
- **Non-parametric knowledge:** Information that is stored externally, for example, in a vector database.
e.

![RAG](RAG.jpg)

The RAG workflow consists of:

1. **The Retrieve**: The user query is used to retrieve relevant context from an external knowledge source. For this, the user query is embedded using an embedding model into the same vector space as the additional context in the vector database. This enables a similarity search, and the top k closest data objects from the vector database are returned.
2. **Augment**: The user query and the retrieved additional context are incorporated into a prompt template.
3. **Generate**: Finally, the retrieval-augmented prompt is fed to the LLM.

We will use the `langchain` framework to efficiently prompt the LLMs and prepare the RAG.


In [3]:
import os
import tempfile
from pathlib import Path
import joblib
import requests
import torch
import transformers

In [ ]:
!pip install --no-cache-dir "pydantic>=2.0"

In [1]:
import pydantic
print(pydantic.__version__)


2.11.3


In [4]:
from IPython.display import display_markdown
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter, TokenTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.vectorstores.faiss import FAISS

In [5]:
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
device = f"cuda:{torch.cuda.current_device()}"

Large Language Models are known for their significant computational demands. Typically, the size of a model is determined by multiplying the number of parameters (size) by the precision of these values (data type). To conserve memory, weights can be stored using lower-precision data types through a process known as quantization.

**Post-Training Quantization (PTQ)** is a straightforward technique where the weights of an already trained model are converted to a lower precision without necessitating any retraining. Although easy to implement, PTQ can lead to potential performance degradation.We will employ PTQ using the `bitsandbytes` library and will load the model in 4-bit precision, applying double quantization with the `nf4` data type. For more information about quantization, visit [this guide on quantization](https://huggingface.co/docs/optimum/en/concept_guides/quantization) )pe.


In [6]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True, # loading in 4 bit
    bnb_4bit_quant_type="nf4", # quantization type
    bnb_4bit_use_double_quant=True, # nested quantization
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [7]:
!huggingface-cli login --token <INSERT_TOKEN>

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `jupyter-notebook` has been saved to C:\Users\Admin\.cache\huggingface\stored_tokens
Your token has been saved to C:\Users\Admin\.cache\huggingface\token
Login successful.
The current active token is: `jupyter-notebook`


In [8]:
model_config = transformers.AutoConfig.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
    config=model_config,
    quantization_config=bnb_config, # we introduce the bnb config here.
    device_map="auto",
)
model.eval()

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

C:\Users\Admin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

We also need to load the tokenizer to transform the text as before.

In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

We will use pipelines from Hugging Face to perform the prompting and generation with the Mistral model.


In [10]:
generate_text = transformers.pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    #temperature=0.0,
    max_new_tokens=8192,
    repetition_penalty=1.1,
)

Device set to use cuda:0


We will use `langchain` to link the HuggingFace models and the chaining prompting.

In [11]:
llm = HuggingFacePipeline(pipeline=generate_text)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11056\3495189882.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generate_text)


The core functionality of `langchain` is the creation of templates for prompting via `PromptTemplate`.

In [12]:
from langchain_core.prompts import PromptTemplate

In [13]:
template = """
You are a helpful AI QA assistant. When answering questions, use the context enclosed by triple backquotes if it is relevant.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Reply your answer in markdown format.
Question: {question}
Answer:"""
prompt = PromptTemplate.from_template(template)

In [14]:
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="\nYou are a helpful AI QA assistant. When answering questions, use the context enclosed by triple backquotes if it is relevant.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nReply your answer in markdown format.\nQuestion: {question}\nAnswer:")

In [15]:
chain = prompt | llm

In [16]:
chain

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="\nYou are a helpful AI QA assistant. When answering questions, use the context enclosed by triple backquotes if it is relevant.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nReply your answer in markdown format.\nQuestion: {question}\nAnswer:")
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x00000261D33FD410>)

In [17]:
question = "What is the scoring criteria of the NLP course?"

In [18]:
print(chain.invoke({"question": question}))

C:\Users\Admin\anaconda3\Lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(



You are a helpful AI QA assistant. When answering questions, use the context enclosed by triple backquotes if it is relevant.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Reply your answer in markdown format.
Question: What is the scoring criteria of the NLP course?
Answer: I'd be happy to help you with that! However, without specific context regarding which NLP (Natural Language Processing) course you're referring to, I can only provide some general information about common scoring criteria used in NLP courses.
Here are some common assessment areas for NLP projects and assignments:
- **Accuracy**: This measures how often the model or algorithm produces correct results.
- **Precision**: This measures the proportion of true positives among the total number of positive predictions made by the model.
- **Recall**: This measures the proportion of true positives among all actual positive instances in the data.
- **F1 Score**: This is the harmo

In [ ]:
question = "What do I have to do for the peer review?"

In [38]:
!pip show huggingface_hub --version

Name: huggingface-hub
Version: 0.30.2
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: C:\Users\Admin\anaconda3\Lib\site-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: accelerate, datasets, sentence-transformers, tokenizers, transformers


In [ ]:
print(chain.invoke({"question": question}))

**NOTE**: The model provides responses that are fairly general. We plan to enhance this by integrating contextual details from our course materials via **RAG**.

### Enter RAG

Next, we will define the vector embeddings of our context. We will use the `sentence-transformers/all-mpnet-base-v2` model to embed the documents and a FAISS vector store to store and later retrieve them. LangChain offers the `HuggingFaceEmbeddings` interface to easily load any model from Hugging Face to serve as the document representation.


In [19]:
EMBED_MODEL = "sentence-transformers/all-mpnet-base-v2"

In [20]:
embedding = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": "cuda"},
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11056\2346813208.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Admin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
def fetch_websites(sites: list[str]):
    docs = []
    with tempfile.TemporaryDirectory() as tmpdir:
        filename = f"{tmpdir}/site.html"
        for site in sites:
            res = requests.get(site)
            with open(filename, mode="wb") as fp:
                fp.write(res.content)
            docs.extend(BSHTMLLoader(filename, open_encoding = 'utf8').load())
    return docs

In [22]:
course_webpage = "https://docs.google.com/document/d/e/2PACX-1vRwReAm5Vz_aiNyZN33eTz22fqMJhM0H-KtJdXthUw5cean_WYdBkZgJchP_s9th0rtUW0ikZZ_Fh5l/pub"

The `fetch_websites` function will be used to scrape data from our Google Document.


In [23]:
docs = fetch_websites([course_webpage])

In [24]:
docs[0]

Document(metadata={'source': 'C:\\Users\\Admin\\AppData\\Local\\Temp\\tmpyadichtx/site.html', 'title': 'Natural language processing 2025'}, page_content='Natural language processing 2025Opublikowano za pomocą Dokumentów GoogleZgłoś nadużycieWięcej informacjiNatural language processing 2025Automatyczne aktualizacje co 5\xa0minLaboratory work - Spring 2025The main goal of laboratory work is to present the most important aspects of data science in practice and to teach you how to use key tools for a NLP engineer. We especially emphasize on self-paced work, raising standards related to development, replicability, reporting, research, visualizing, etc. Our goal is not to provide exact instructions or "make robots" out of participants of this course. Participants will need to try to navigate themselves among data, identify promising leads and extract as much information as possible from the data to present to the others (colleagues, instructors, companies or their superiors).Important linksL

We loaded the entire course into a single document. Since the sentence transformer can handle only limited sections of text, this might be problematic. Next, we will use the `RecursiveCharacterTextSplitter` to split the document into chunks with a `chunk_size` of 1000 characters and a `chunk_overlap` of 20.


In [25]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

Let's see what the chunked document looks like.


In [ ]:
all_splits[0]

In [ ]:
all_splits[1]

In [ ]:
all_splits[2]

Next, we initialize a vector store. A vector store is a data structure that functions as a vector database, where each document is stored based on its own embedding. We will use the `FAISS` library for this purpose.


In [ ]:
vectorstore = FAISS.from_documents(all_splits, embedding)

Finally, we define a retriever—an object that will handle the **retrieving** part of the RAG pipeline. The retriever receives as arguments the metric by which we search the space and the number of the k-nearest documents (chunks in our case) that we retrieve to present to the studen


In [ ]:
retreiver = vectorstore.as_retriever(
    search_type="similarity",
    k=10,
)

Next, we modify our prompt template so that it now receives the context (the documents selected by the RAG system) and then generates the answer.


In [ ]:
PROMPT_TEMPLATE = """
You are a helpful AI QA assistant. When answering questions, use the context enclosed by triple backquotes if it is relevant.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Reply your answer in markdown format.

```
{context}
```

### Question:
{question}

### Answer:
"""

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=PROMPT_TEMPLATE.strip(),
)

With the prompt defined, we next set up the `ConversationalRetrievalChain` that will utilize the defined `retriever` and `llm`, following the `PROMPT_TEMPLATE` to extract documents.


In [ ]:
# Construct complete LLM chain
llm_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retreiver,
    return_source_documents=False,
    combine_docs_chain_kwargs={"prompt": prompt_template},
    verbose=False,
)

Finally, we create the `answer_question` function that will handle the chain invocation for us.


In [ ]:
def answer_question(question: str, history: dict[str] = None) -> str:
    if history is None:
        history = []

    response = llm_chain.invoke({"question": question, "chat_history": history})
    answer = response["answer"].split("### Answer:")[-1].strip()
    return answer

In [ ]:
question = "What do I have to do for the peer review?"
display_markdown(answer_question(question), raw=True)

In [ ]:
question = "What is the first project about?"
display_markdown(answer_question(question), raw=True)

In [ ]:
question = "What do you have to do for a good grade?"
display_markdown(answer_question(question), raw=True)

In [ ]:
question = "What is the project intended for digital linguistics about?"
display_markdown(answer_question(question), raw=True)

With the introduction of RAG, the `Mistral` model was able to successfully answer questions about the NLP course.

**Exercises:**
* Implement a data loader script that can load documents from a folder.
